# Construct Knowledge Graph

In [39]:
from py2neo import Graph, Node, Relationship
import spacy
import pandas as pd
import numpy as np


username="neo4j"
password="neo4j"

graph = Graph("bolt://localhost:7687", auth=(username, password)) # Connect to the graph database

# Create nodes like this
def create_nodes(nodelabel, nodelist):
    for nodename in nodelist:
        node = Node(nodelabel, label=nodename, text=nodename, name=nodename)
        graph.create(node)


def create_relationships(df, graph):
    for _, row in df.iterrows():
        character_a = Node("Character", label="Character", text=row["Character"])
        character_b = Node("Character", label="Character", text=row["Target Character"])
        graph.merge(character_a, "Character", "text")
        graph.merge(character_b, "Character", "text")
        relationship = Relationship(character_a, row["Relationship Type"], character_b, notes=row["Notes"])
        graph.create(relationship)


def delete_all(graph, nodelabel="Character"):
    graph.run(f"MATCH (n: `{nodelabel}`) DETACH DELETE n")

def get_all_characters(graph):
    query = "MATCH (n:Character) RETURN n.text"
    result = graph.run(query).to_data_frame()
    return result

def get_relationships_by_character(graph, character):
    query = """
    MATCH (c:Character)-[r]-(t:Character) 
    WHERE c.text = $character_name
    RETURN c.text AS Character, type(r) AS RelationshipType, t.text AS TargetCharacter, r.notes AS Notes
    """
    result = graph.run(query, character_name=character).to_data_frame()
    return result

# delete_all(graph)

# Initiallize Nodes

In [26]:
df = pd.read_csv("Summary/character_relationships.csv")
charactersA = df["Character"].unique()
charactersB = df["Target Character"].unique()
characters = list(set(np.concatenate([charactersA, charactersB])))
create_nodes("Character", characters)    

![Character](https://cdn.jsdelivr.net/gh/AllenYGY/ImageSpace@main/uPic/Character.png)

# Construct node relationships

In [45]:
create_relationships(df, graph)

![Knowledge Graph](https://cdn.jsdelivr.net/gh/AllenYGY/ImageSpace@main/uPic/3chsG1.png)


# Get all Characters


In [44]:
get_all_characters(graph)

,n.text
0,Albus Dumbledore
1,Professor Snape
2,Hogwarts students
3,Firenze
4,Neville Longbottom
5,"Ron Weasley, Fred Weasley, George Weasley"
6,Severus Snape
7,Lord Voldemort
8,Fluffy
9,"Harry Potter, Ron Weasley"


In [47]:
get_relationships_by_character(graph, "Ron Weasley")

,Character,RelationshipType,TargetCharacter,Notes
0,Ron Weasley,Friends with,Hermione Granger,School friend
1,Ron Weasley,Gryffindor House,Harry Potter,House member
2,Ron Weasley,Student of,Professor Snape,Teacher in Potions
3,Ron Weasley,Student of,Professor McGonagall,Teacher in Transfiguration
4,Ron Weasley,Sibling of,Fred Weasley,Family relationship
5,Ron Weasley,Sibling of,Ginny Weasley,Family relationship
6,Ron Weasley,Friends with,Hermione Granger,School friend
7,Ron Weasley,Friends with,Harry Potter,Close school friend
8,Ron Weasley,Friends with,Harry Potter,Close school friend and Gryffindor
